In [10]:
from ioMicroMosaic import *

def get_mosaic_image_T(data_fld,resc = 4,icol = 1,frame = 20,force=False,rescz=2):
    save_fld = os.path.dirname(data_fld)+os.sep+'mosaics'
    if not os.path.exists(save_fld): os.makedirs(save_fld)
    fl_save = save_fld+os.sep+os.path.basename(data_fld)+'_frame'+str(frame)+'_col'+str(icol)+'.tiff'
    print(fl_save)
    if not os.path.exists(fl_save) or force:
        fls_ = np.sort(glob.glob(data_fld+r'\*.zarr'))
        ims,xs_um,ys_um=[],[],[]

        for fl in tqdm(fls_[:]):
            im,x,y = read_im(fl,return_pos=True)
            
            if str(frame).lower()=='all':
                ims.append(np.array(np.max(im[icol][::rescz,::resc,::resc],axis=0),dtype=np.float32))
            else:
                ims.append(np.array(im[icol][frame,::resc,::resc],dtype=np.float32))
            xs_um.append(x)
            ys_um.append(y)
        ims_ = [im[::1,::-1].T for im in ims]
        im_big,xs,ys = compose_mosaic(ims_,xs_um,ys_um,ims_c=None,
                                      um_per_pix=0.1083333*resc,
                                      rot = 0,return_coords= True)
        
        import tifffile
        tifffile.imwrite(fl_save,im_big)
        
        
        resc_ = 3
        data_fld = os.path.dirname(fls_[0])
        fig = plt.figure(figsize=(30,30))
        im__ = im_big[::resc_,::resc_]
        #if vmax is None:
        vmax = np.percentile(im__[im__>0],99.)
        vmin=np.percentile(im__[im__>0],1.)
        plt.imshow(im_big.T[::resc_,::resc_],vmin=vmin,vmax=vmax,cmap='gray')
        #fig.savefig(fl_save)
        for x_,y_,fl_ in zip(xs,ys,fls_):
            ifov = fl_.split('_')[-1].split('.')[0]
            plt.text(x_/resc_,y_/resc_,ifov,color='r')
        plt.xticks([])
        plt.yticks([])
        fl_save = fl_save.replace('.tiff','_annot.png')
        fl_saveT = fl_save.replace('.tiff','.npz')
        np.savez(fl_saveT,fls=fls_,xs=xs,ys=ys,xs_um=xs_um,ys_um=ys_um)
        print(fl_save)
        fig.savefig(fl_save)
        plt.close('all')

In [26]:
#flds = glob.glob(r'C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2*')+glob.glob(r'D:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2*')
#flds = glob.glob(r'D:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set2') + glob.glob(r'D:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set4') + glob.glob(r'C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set1') + glob.glob(r'C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set3')+ glob.glob(r'C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set5')
flds = glob.glob(r'C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set11')+glob.glob(r'D:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set12')+glob.glob(r'C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set13')+glob.glob(r'D:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set14')+glob.glob(r'C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\H2_set15')
flds = np.array(flds)[np.argsort([os.path.getmtime(fld)for fld in flds])]
flds

array(['C:\\Qiao\\NB1-7-13-10_13_2024_FL0R0_10\\H2_set11',
       'D:\\Qiao\\NB1-7-13-10_13_2024_FL0R0_10\\H2_set12',
       'C:\\Qiao\\NB1-7-13-10_13_2024_FL0R0_10\\H2_set13',
       'D:\\Qiao\\NB1-7-13-10_13_2024_FL0R0_10\\H2_set14',
       'C:\\Qiao\\NB1-7-13-10_13_2024_FL0R0_10\\H2_set15'], dtype='<U45')

In [28]:
for data_fld in flds:
    get_mosaic_image_T(data_fld,resc = 6,icol = 0,frame = 'all',force=True,rescz=2)
    get_mosaic_image_T(data_fld,resc = 6,icol = 1,frame = 'all',force=True,rescz=2)

C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\mosaics\H2_set11_frameall_col2.tiff


  0%|                                                                                          | 0/418 [00:00<?, ?it/s]


IndexError: Index 2 is out of bounds for axis 0 with size 2

In [17]:
get_mosaic_image_T(data_fld,resc = 4,icol = 0,frame = 3,force=False,rescz=2)

C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\mosaics\H1_set1_frame3_col0.tiff


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.13it/s]


C:\Qiao\NB1-7-13-10_13_2024_FL0R0_10\mosaics\H1_set1_frame3_col0_annot.png
